# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [11]:
# imports

from openai import OpenAI
from dotenv import load_dotenv
import os
from IPython.display import Markdown, update_display

load_dotenv()


True

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment

client = OpenAI(base_url='https://api.deepseek.com', api_key=os.getenv('DEEPSEEK_API_KEY'))

In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [10]:
# Get gpt-4o-mini to answer, with streaming

stream = client.chat.completions.create(
    model='deepseek-chat',
    messages=[
        {"role": "system", "content": "You are a helpful assistant that can answer questions about the code."},
        {"role": "user", "content": question}
    ],
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

This code uses a generator expression with `yield from` to yield all unique authors from a list of books where the author field exists and is non-empty. Let's break it down:

1. `{book.get("author") for book in books if book.get("author")}` is a set comprehension that:
   - Iterates through each `book` in the `books` list
   - Uses `book.get("author")` to safely get the "author" field (returns None if key doesn't exist)
   - Only includes authors that are truthy (not None, empty string, etc.) due to the `if` condition
   - Creates a set (which automatically deduplicates authors)

2. `yield from` then takes this set of unique authors and yields each one individually from the generator.

So the overall effect is that this code will produce a generator that yields each unique author name from the books list, skipping any books that don't have an author specified.

Example:
python
books = [
    {"title": "Book1", "author": "AuthorA"},
    {"title": "Book2", "author": "AuthorB"},
    {"title": "Book3"},  # no author
    {"title": "Book4", "author": "AuthorA"},  # duplicate author
    {"title": "Book5", "author": ""},  # empty author
]

def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

list(get_authors(books))  # Returns ['AuthorA', 'AuthorB']


Note: Using a set means the order of authors is not preserved. If order matters, you could use a dict or list comprehension instead.

In [13]:
# Get Llama 3.2 to answer
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

messages = [
    {"role": "user", "content": question}
]

response = ollama_via_openai.chat.completions.create(
    model='llama3.2',
    messages=messages
)

print(response.choices[0].message.content)

This line of code uses a combination of Python's `yield` keyword, `from` keyword, dictionary (`dict`), list (`list`), and conditional expressions.

Here's a breakdown:

- **`yield from`**: This syntax was introduced in Python 3.3 as part of PEP 405 (Yield From Expression). It allows you to yield values from another iterable or generator, which is more memory-efficient than creating a new list and returning it explicitly.

- `{book.get("author") for book in books if book.get("author")}`: This is a dictionary comprehension generator expression. 

    - `for book in books`: Iterates through each item (`book`) in the 'books' iterable.
    
    - `if book.get("author")`: Applies a condition to only include items where `'author'` key exists and has a non-empty value.

- `yield from {expression}`: When used with dictionary comprehension generators, it yields values in an efficient manner rather than returning them as a whole list (which would consume significant memory when dealing with large